<a href="https://colab.research.google.com/github/haytham918/low-rank-expectile/blob/main/LRE_ActualData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [24]:
df= pd.read_csv("/content/drive/MyDrive/low-rank-expectile/heartrate_seconds_merged.csv")

# Convert 'Time' column to datetime type
df['Time'] = pd.to_datetime(df['Time'])

# Round each time down to the 5-min interval
df['Time'] = df['Time'].dt.floor('5min')

# Create a new column with just the Date
df['Date'] = df['Time'].dt.date

# Create a new column for the 5-minute interval index
df['Interval'] = df['Time'].dt.strftime('%H:%M')

# Group by 'Id', 'Date', and 'Interval' then calculate median
# Unstack such that level[0] is Id and level[1] is Date
df_median = df.groupby(['Id', 'Date', 'Interval'])['Value'].median().unstack(level=[0,1])

# Reindex to include all 5-minute intervals over a day and missing values with Nan
all_intervals = pd.date_range(start='00:00', end='23:55', freq='5min').strftime('%H:%M')
df_median = df_median.reindex(all_intervals)

df_median.head(200)

Id    2022484408                                                         \
Date  2016-04-12 2016-04-13 2016-04-14 2016-04-15 2016-04-16 2016-04-17   
00:00        NaN        NaN        NaN        NaN        NaN        NaN   
00:05        NaN        NaN        NaN        NaN        NaN        NaN   
00:10        NaN        NaN        NaN        NaN        NaN        NaN   
00:15        NaN        NaN        NaN        NaN        NaN        NaN   
00:20        NaN        NaN        NaN        NaN        NaN        NaN   

Id                                                 ... 8877689391             \
Date  2016-04-18 2016-04-19 2016-04-20 2016-04-21  ... 2016-05-03 2016-05-04   
00:00        NaN        NaN        NaN        NaN  ...        NaN        NaN   
00:05        NaN        NaN        NaN        NaN  ...        NaN        NaN   
00:10        NaN        NaN        NaN        NaN  ...        NaN        NaN   
00:15        NaN        NaN        NaN        NaN  ...        NaN        NaN   
00:20        NaN        NaN        NaN        NaN  ...        NaN        NaN   

Id                                                                       \
Date  2016-05-05 2016-05-06 2016-05-07 2016-05-08 2016-05-09 2016-05-10   
00:00        NaN        NaN        NaN        NaN        NaN        NaN   
00:05        NaN        NaN        NaN        NaN        NaN        NaN   
00:10        NaN        NaN        NaN        NaN        NaN        NaN   
00:15        NaN        NaN        NaN        NaN        NaN        NaN   
00:20        NaN        NaN        NaN        NaN        NaN        NaN   

Id                           
Date  2016-05-11 2016-05-12  
00:00        NaN        NaN  
00:05        NaN        NaN  
00:10        NaN        NaN  
00:15        NaN        NaN  
00:20        NaN        NaN  

[5 rows x 334 columns]

In [56]:
# Convert the df to numpy matrix
user_times_matrix = df_median.values

# Nan Mask Matrix for the matrix
nan_mask = np.isnan(user_times_matrix)

number_missing = np.sum(nan_mask)
matrix_shape = user_times_matrix.shape
print("Matrix shape is :", matrix_shape)
print("Number of Missing values: ", number_missing)
print("Proportion of Missing values: ", number_missing / (matrix_shape[0] * matrix_shape[1]) * 100, "%")

# Check if there is any interval that there is no data from any user
print("Index of the interval where there is no data at all: ", np.where(nan_mask.sum(axis=1) == matrix_shape[1]))
# Get the masked matrix so that we ignore nan values
masked_matrix = np.ma.array(user_times_matrix, mask=nan_mask)


Matrix shape is : (288, 334)
Number of Missing values:  28674
Proportion of Missing values:  29.809131736526943 %
Index of the interval where there is no data at all:  (array([], dtype=int64),)
